In [2]:
import pennylane as qml
import numpy as np
from itertools import combinations

# Define the quantum device
n_qubits = 12
q_depth = 10
dev = qml.device("default.qubit", wires=n_qubits)

# === 1. Quantum Circuit Components ===
def H_layer(n_qubits):
    """Layer of single-qubit Hadamard gates."""
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)


def RY_layer(params):
    """Layer of parametrized qubit rotations around the y-axis."""
    for idx, param in enumerate(params):
        qml.RY(param, wires=idx)


def entangling_layer(n_qubits):
    """Layer of CNOT gates for Net1."""
    for i in range(0, n_qubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, n_qubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])


def U3_layer(n_qubits):
    """Layer of single-qubit U3 rotations for Net3."""
    params = np.random.uniform(0, 2 * np.pi, (n_qubits, 3))
    for idx in range(n_qubits):
        qml.U3(params[idx][0], params[idx][1], params[idx][2], wires=idx)


def phase_layer(n_qubits):
    """Layer of phase gates for Net3."""
    for idx in range(n_qubits):
        if idx % 2 == 0:
            qml.S(wires=idx)
        else:
            qml.T(wires=idx)


def entangling_layer_3(n_qubits):
    """Layer of controlled-Rz gates followed by zz entangling gates for Net3."""
    params = np.random.uniform(0, 2 * np.pi, (n_qubits,))
    for i in range(0, n_qubits - 1, 2):
        qml.ctrl(qml.RZ, control=i)(params[i], wires=i + 1)
    for i in range(1, n_qubits - 1, 2):
        qml.IsingZZ(params[i], wires=[i, i + 1])


# === 2. Circuit Definitions ===
def net1_circuit(features, weights):
    """Net1 quantum circuit definition."""
    H_layer(n_qubits)  # Apply Hadamard layer
    RY_layer(features)  # Embed input features
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(weights[k])  # Apply parametrized RY layer


def net3_circuit(features, weights):
    """Net3 quantum circuit definition."""
    H_layer(n_qubits)  # Apply Hadamard layer
    for _ in range(q_depth):
        U3_layer(n_qubits)
        phase_layer(n_qubits)
        entangling_layer_3(n_qubits)


# === 3. Unified Quantum Circuit Execution ===
@qml.qnode(dev)
def quantum_circuit(features, weights, circuit_fn):
    """
    A unified quantum circuit executor.
    """
    circuit_fn(features, weights)  # Execute the given circuit
    return qml.state()  # Return the full quantum state


# === 4. CE Computation Functions ===
def state_to_density_matrix(state):
    """Convert a state vector to a density matrix."""
    return np.outer(state, np.conj(state))


def reduced_density_matrix(matrix, subsystem, n_qubits):
    """Compute the reduced density matrix by tracing out all other qubits."""
    all_indices = list(range(n_qubits))
    trace_out_indices = [i for i in all_indices if i not in subsystem]
    return qml.math.partial_trace(matrix, indices=trace_out_indices)


def purity(rho):
    """Compute the purity of a density matrix."""
    return np.real(np.trace(rho @ rho))


def concentratable_entanglement(state, qubits, n_qubits):
    """
    Compute the Concentratable Entanglement (CE) for a subset of qubits.
    """
    density_matrix = state_to_density_matrix(state)
    purities = []
    for q in qubits:
        rho_q = reduced_density_matrix(density_matrix, [q], n_qubits)
        purities.append(purity(rho_q))
    return 1 - np.mean(purities)


def compute_ce(features, weights, circuit_fn, subset_size, n_qubits):
    """
    Compute CE for a given quantum circuit.
    """
    # Get quantum state
    state = quantum_circuit(features, weights, circuit_fn)

    # Compute CE for all subsets of the given size
    subsets = list(combinations(range(n_qubits), subset_size))
    ce_values = {}
    for subset in subsets:
        ce = concentratable_entanglement(state, subset, n_qubits)
        ce_values[subset] = ce

    return ce_values


# === 5. Main Program ===
if __name__ == "__main__":
    # Generate random input features and weights
    features = np.random.uniform(0, np.pi, size=n_qubits)
    weights_net1 = np.random.uniform(0, np.pi, size=(q_depth, n_qubits))
    weights_net3 = np.random.uniform(0, np.pi, size=(q_depth, n_qubits))

    # Compute CE for Net1
    ce_net1 = compute_ce(features, weights_net1, net1_circuit, subset_size=12, n_qubits=n_qubits)
    print("\nConcentratable Entanglement for Net1:")
    for subset, ce in ce_net1.items():
        print(f"Subset {subset}: CE = {ce:.4f}")

    # Compute CE for Net3
    ce_net3 = compute_ce(features, weights_net3, net3_circuit, subset_size=12, n_qubits=n_qubits)
    print("\nConcentratable Entanglement for Net3:")
    for subset, ce in ce_net3.items():
        print(f"Subset {subset}: CE = {ce:.4f}")



Concentratable Entanglement for Net1:
Subset (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11): CE = 0.4968

Concentratable Entanglement for Net3:
Subset (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11): CE = 0.4199
